### 1. Модель на основе LSTM
Для оценки работоспособности кода в папке `data` находится датасет, усечённый до 50000 строк.   
Для оценки работы предварительно обученной модели, можно перейти в блок 8 и далее.  

Модель обучалась на 12 эпохах.  
Гиперпараметры модели:  
```
EMBEDDING_DIM = 128  
HIDDEN_DIM = 512  
NUM_LAYERS = 1  
```
Размер батча - 256  
  
Epoch 1 | Train Loss: 5.439 | Val Loss: 5.311 | Val PPL: 202.47 | Val Accuracy: 17.53%  
Epoch 2 | Train Loss: 5.327 | Val Loss: 5.291 | Val PPL: 198.52 | Val Accuracy: 17.64%  
Epoch 3 | Train Loss: 5.314 | Val Loss: 5.285 | Val PPL: 197.29 | Val Accuracy: 17.71%  
Epoch 4 | Train Loss: 5.309 | Val Loss: 5.281 | Val PPL: 196.49 | Val Accuracy: 17.73%  
Epoch 5 | Train Loss: 5.305 | Val Loss: 5.281 | Val PPL: 196.63 | Val Accuracy: 17.75%  
Epoch 6 | Train Loss: 5.303 | Val Loss: 5.279 | Val PPL: 196.10 | Val Accuracy: 17.75%  
Epoch 7 | Train Loss: 5.302 | Val Loss: 5.277 | Val PPL: 195.72 | Val Accuracy: 17.79%  
Epoch 8 | Train Loss: 5.300 | Val Loss: 5.277 | Val PPL: 195.71 | Val Accuracy: 17.76%  
Epoch 9 | Train Loss: 5.300 | Val Loss: 5.277 | Val PPL: 195.74 | Val Accuracy: 17.74%  
Epoch 10 | Train Loss: 5.299 | Val Loss: 5.275 | Val PPL: 195.48 | Val Accuracy: 17.80%  
Epoch 11 | Train Loss: 5.298 | Val Loss: 5.274 | Val PPL: 195.29 | Val Accuracy: 17.75%  
Epoch 12 | Train Loss: 5.298 | Val Loss: 5.274 | Val PPL: 195.15 | Val Accuracy: 17.78%  

Метрика ROUGE на протяжении всего обучения менялась от 0.1333 до 0.2  
Для ускорения расчёта ROUGE, замер происходил на ограниченном количестве 
ROUGE-1: 0.2000  
ROUGE-L: 0.2000  
ROUGE-Lsum: 0.2000  

Несмотря на большие значения loss, наблюдается её падение. Также наблюдается небольшая разница между train loss и val loss, что говорит об остутствии переобучения.  

Было принято решение так же оценить метрику PPL (перплексия), которая показывает неопределённость модели в предсказании следующего токена в последовательности.  
Чем меньше значение PPL, тем меньше неопределённости.  

Несмотря на невысокие показатели метрик, модель генерирует связный текст.  


Пример генерации (автодополнения):  
Исходный: `cant wait to get a car walking in`  
Дополненный: `cant wait to get a car walking in with the family and i have to go to work`  
Кроме детерминированного предсказания, было реализовано предсказание с сэмплированием.  
Пример результата:  
Исходный: `cant wait to get a car walking in`  
Дополненный: `cant wait to get a car walking in with that in my house that has work on saturday`  


![График Loss и Accuracy](loss_and_accuracy.png)  
![График PPL и Accuracy](ppl_and_accuracy.png)

### 2. Модель GPT2 (distilgpt2)
Модель хорошо справляется с генерацией небольших текстов, о чём свидетельствуют неплохие, по сравнению с LSTM, метрики ROUGE.  

ROUGE-1: 0.5697  
ROUGE-2: 0.5048  
ROUGE-L: 0.5697  
ROUGE-Lsum: 0.5697  
  
Пример генерации:  
Промпт: `just got done watching august`  
Референс: `just got done watching august rush love it sooo much`  
Генерация: `just got done watching august.com for a few hours.`  


=======================================================================================================================================================

1. Импорт библиотек, классов и экземпляров классов.

In [ ]:
import torch
from torch.utils.data import DataLoader

from configs.config import settings
from src.data_utils import DataUtils
from src.next_token_dataset import NextTokenDataset, tokenizer

2. Загрузка и очистка входных текстовых данных.  
Разделение данных на выборки.  
Сохранение токенизированных текстов в csv файлы.  

При повторных расчётах, если csv файлы уже были сформированы, можно перейти сразу в блок 3.2 для формирования DataLoader'ов.

In [ ]:
data_utils = DataUtils()
data_utils.load_and_clean('data/tweets50000.txt')
# data_utils.load_and_clean('data/tweets.txt')
data_utils.texts_stats()
data_utils.save_cleaned_data(data_utils.data, 'data/cleaned_tweets.txt')
print(data_utils.data[-1])

train, val, test = data_utils.split_data(data_utils.data)
data_utils.save_cleaned_data(train, 'data/train.txt')
data_utils.save_cleaned_data(val, 'data/val.txt')
data_utils.save_cleaned_data(test, 'data/test.txt')
all_texts = NextTokenDataset(data_utils.data, tokenizer, seq_len=settings.SEQ_LEN)

train_tokens, val_tokens, test_tokens = data_utils.split_data(all_texts.tokens)

data_utils.save_tokenized_data_csv(train_tokens, 'data/train_tokens.csv')
data_utils.save_tokenized_data_csv(val_tokens, 'data/val_tokens.csv')
data_utils.save_tokenized_data_csv(test_tokens, 'data/test_tokens.csv')

print(all_texts[-1])

3. Формирование датасетов и даталоадеров из очищенных данных.  
3.1 Первый блок ипользуется для формирования DataLoader'ов на основе текстов.  

In [ ]:
# на основе текстов
# тренировочный, валидационный и тестовые датасеты из очищенных данных
train_dataset = NextTokenDataset(train, tokenizer, seq_len=settings.SEQ_LEN)
val_dataset = NextTokenDataset(val, tokenizer, seq_len=settings.SEQ_LEN)
test_dataset = NextTokenDataset(test, tokenizer, seq_len=settings.SEQ_LEN)
print("Datasets complete!")
# даталоадеры
train_loader = DataLoader(train_dataset, batch_size=settings.BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=settings.BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=settings.BATCH_SIZE)

print(f"Train dataset size: {len(train_dataset)}, Val dataset size: {len(val_dataset)}, Test dataset size: {len(test_dataset)}")
print(f"Train loader size: {len(train_loader)}, Val loader size: {len(val_loader)}, Test loader size: {len(test_loader)}")

3.2 Второй блок ипользуется для формирования DataLoader'ов на основе токенов, предварительно сохранённых в csv файлы.
  

In [ ]:
# на основе файлов с токенами
train = val = test = []
# тренировочный, валидационный и тестовые датасеты из подготовленных токенов
train_dataset = NextTokenDataset(train, tokenizer, seq_len=settings.SEQ_LEN, tokens_csv='data/train_tokens.csv')
val_dataset = NextTokenDataset(val, tokenizer, seq_len=settings.SEQ_LEN, tokens_csv='data/val_tokens.csv')
test_dataset = NextTokenDataset(test, tokenizer, seq_len=settings.SEQ_LEN, tokens_csv='data/test_tokens.csv')
print("Datasets complete!")
# даталоадеры
train_loader = DataLoader(train_dataset, batch_size=settings.BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=settings.BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=settings.BATCH_SIZE)

print(f"Train dataset size: {len(train_dataset)}, Val dataset size: {len(val_dataset)}, Test dataset size: {len(test_dataset)}")
print(f"Train loader size: {len(train_loader)}, Val loader size: {len(val_loader)}, Test loader size: {len(test_loader)}")

4. Загрузка экземпляра модели, устройства обработки.  
Вывод параметров модели.

In [ ]:
from src.lstm_model import model, device, count_parameters

param_count = count_parameters(model)
print(f"{param_count:>10,}")

5. Загрузка экземпляра класса оценки модели.  
Параметр compute_rouge указывает на необходимость расчёта метрики ROUGE.

In [ ]:
from src.eval_lstm import EvalLSTM

evaluator = EvalLSTM(model, device, compute_rouge=True)

6. Загрузка и запуск экземпляра класса обучения модели

In [ ]:
from src.lstm_train import LSTMTrain

lstm = LSTMTrain(model, device, evaluator)
metrics_history = lstm.model_train(train_loader, val_loader, tokenizer)

7. Строим графики метрик

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 5)
plt.rcParams['font.size'] = 10

df = pd.DataFrame(metrics_history)
fig, ax1 = plt.subplots()

ax1.plot(df['epoch'], df['train_loss'], label='Train Loss', color='blue', marker='o')
ax1.plot(df['epoch'], df['val_loss'], label='Val Loss', color='red', marker='s')
ax1.plot(df['epoch'], df['val_ppl'], label='Val PPL', color='orange', marker='v')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss', color='black')
ax1.tick_params(axis='y', labelcolor='black')
ax1.grid(True, alpha=0.3)

ax2 = ax1.twinx()
ax2.plot(df['epoch'], df['val_accuracy'], label='Val Accuracy', color='green', marker='^', linestyle='--')
ax2.set_ylabel('Accuracy', color='green')
ax2.tick_params(axis='y', labelcolor='green')

lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper right')

plt.title('Loss and Accuracy over Epochs')
plt.tight_layout()
plt.show()

8. Загрузка предварительно обученной модели

In [ ]:
# Загрузка обученной модели
model.load_state_dict(torch.load(f"{settings.MODEL_PATH}_{settings.HIDDEN_DIM}_{settings.NUM_LAYERS}layer_state_dict.pth"))
model.eval()

9. Оценка работы генерации твитов.

In [ ]:
# примеры твитов из тестовой выборки
'''
still trying to figure out the purpose of twitter bah headed back to beautiful bryan texas
just got done watching august rush love it sooo much
oliyoung fair enough well ill have a crack at it over the weekend where did you get your formulae from
cant wait to get a car walking in the cold is seriously not fun anymore
'''

prompt = "cant wait to get a car walking in"
input_ids = tokenizer.encode(prompt)

text = model.generate(input_ids, n_tokens=10, tokenizer=tokenizer)
print(f"{prompt} {text}")

In [ ]:
prompt = "cant wait to get a car walking in"
input_ids = tokenizer.encode(prompt)

text = model.generate(input_ids, n_tokens=10, tokenizer=tokenizer, sampling=True)
print(f"{prompt} {text}")

10. Сохранение модели

In [ ]:
torch.save(model.state_dict(), f"{settings.MODEL_PATH}_{settings.HIDDEN_DIM}_{settings.NUM_LAYERS}layer_state_dict.pth")


11. Генерация текста с помощью трансформера.  
Модель на основе GPT2

In [ ]:
from src.eval_transformer_pipeline import EvalTransformer

gpt_model = "distilgpt2"
evaluator = EvalTransformer(model_name=gpt_model)

dataset_prompts = [
    "still trying to figure out the purpose of twitter",
    "oliyoung fair enough well ill have a crack at it",
    "just got done watching august"
]

dataset_references = [
    "still trying to figure out the purpose of twitter bah headed back to beautiful bryan texas",
    "oliyoung fair enough well ill have a crack at it over the weekend where did you get your formulae from",
    "just got done watching august rush love it sooo much"
]

# Параметры генерации
generation_params = {
    "max_length": 20,  # промпт + ответ
    "num_return_sequences": 1,
    "do_sample": True,
    "temperature": 0.8,
    "top_p": 0.95,
    "no_repeat_ngram_size": 2
}

# Запуск валидации
results = evaluator.validate(
    prompts=dataset_prompts,
    references=dataset_references,
    generation_config=generation_params
)

# Вывод результатов
print('\n--- Результаты метрик ---')
for k, v in results.items():
    print(f"{k}: {v:.4f}")

print('\n--- Пример генерации ---')
sample_out = evaluator.generate_texts([dataset_prompts[2]], **generation_params)
print(f"Промпт: {dataset_prompts[2]}")
print(f"Референс: {dataset_references[2]}")
print(f"Генерация: {sample_out[0]}")
